In [ ]:
# Required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras

In [ ]:
# Split a univariate sequence into samples for LSTM model
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Load the dataset
filepath_content = ".../BCN-HRB_Points.txt"
df_Lis = pd.read_csv(filepath_content, sep="\t", header=None, index_col=0)
dataset_df = df_Lis.iloc[:, 10:]  # Selecting relevant columns (Time Series data)
len_TS = dataset_df.shape[1]
Anomaly_Period = 10
len_input = len_TS - Anomaly_Period

In [ ]:
# Dataset for training and testing
dataset = dataset_df.iloc[:, Anomaly_Period:]
test_dataset = dataset.iloc[1800:, :]

# Transpose and prepare data for LSTM model
dataset_t = dataset.transpose().values.astype('float32')
test_dataset_t = test_dataset.transpose().values.astype('float32')

In [ ]:
# Parameters for the model
n_steps_in, n_steps_out = 10, 1

# Split dataset into input/output sequences for the model
X, y = split_sequences(dataset_t, n_steps_in, n_steps_out)

In [ ]:
# Reshape input and output data
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [ ]:
# LSTM model parameters
learning_rate = 0.1
batch_size = 100
epochs = 10
units = 32

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Dense(units, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
opt = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt, loss='mae')

In [ ]:
# Train the model
history = model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2)

In [ ]:
# Evaluate the model
lossT = model.evaluate(X, y, batch_size=batch_size)
print(f'Loss: {lossT:.5f}')

In [ ]:
# Test data prediction
x_input = test_dataset_t
x_input = x_input.reshape((n_input, 1))
yhat = model.predict(x_input, verbose=2)
print(yhat)